In [1]:
import numpy as np
import pandas as pd

In [3]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder

In [4]:
df = pd.read_csv('covid_toy.csv')

In [5]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [6]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)

In [8]:
X_train

,age,gender,fever,cough,city
28,16,Male,104.0,Mild,Kolkata
13,64,Male,102.0,Mild,Bangalore
92,82,Female,102.0,Strong,Kolkata
71,75,Female,104.0,Strong,Delhi
23,80,Female,98.0,Mild,Delhi
...,...,...,...,...,...
61,81,Female,98.0,Strong,Mumbai
45,72,Male,99.0,Mild,Bangalore
49,44,Male,104.0,Mild,Mumbai
34,74,Male,102.0,Mild,Mumbai


## 1. Aam Zindagi

In [9]:
# adding simple imputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])

X_train_fever.shape

(80, 1)

In [10]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [12]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [43]:
#LabelEncoding --> has_covind
le = LabelEncoder()
y_train_hascovid=pd.DataFrame(le.fit_transform(y_train))
y_test_hascovid=pd.DataFrame(le.fit_transform(y_test))
y_train_hascovid.shape

(80, 1)

In [41]:
le.classes_

array(['No', 'Yes'], dtype=object)

In [21]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [74]:
X_train_transformed = pd.DataFrame(np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1))
# also the test data
X_test_transformed = pd.DataFrame(np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1))

X_train_transformed.shape

(80, 7)

In [78]:
#Final_add all
final_train_data = pd.concat((X_train_transformed,y_train_hascovid))
final_test_data =pd.concat((X_test_transformed,y_test_hascovid))

final_train_data.shape

(160, 7)

In [79]:
final_train_data

,0,1,2,3,4,5,6
0,16.0,104.0,1.0,0.0,1.0,0.0,0.0
1,64.0,102.0,1.0,0.0,0.0,0.0,0.0
2,82.0,102.0,0.0,0.0,1.0,0.0,1.0
3,75.0,104.0,0.0,1.0,0.0,0.0,1.0
4,80.0,98.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
75,0.0,NaN,NaN,NaN,NaN,NaN,NaN
76,0.0,NaN,NaN,NaN,NaN,NaN,NaN
77,0.0,NaN,NaN,NaN,NaN,NaN,NaN
78,1.0,NaN,NaN,NaN,NaN,NaN,NaN


### Mentos Zindagi

In [80]:
from sklearn.compose import ColumnTransformer

In [89]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [90]:
transformer.fit_transform(X_train).shape

(80, 7)

In [91]:
transformer.transform(X_test).shape

(20, 7)

In [94]:
pd.DataFrame(transformer.fit_transform(X_train))

,0,1,2,3,4,5,6
0,104.000000,0.0,1.0,0.0,1.0,0.0,16.0
1,102.000000,0.0,1.0,0.0,0.0,0.0,64.0
2,102.000000,1.0,0.0,0.0,1.0,0.0,82.0
3,104.000000,1.0,0.0,1.0,0.0,0.0,75.0
4,98.000000,0.0,0.0,1.0,0.0,0.0,80.0
...,...,...,...,...,...,...,...
75,98.000000,1.0,0.0,0.0,0.0,1.0,81.0
76,99.000000,0.0,1.0,0.0,0.0,0.0,72.0
77,104.000000,0.0,1.0,0.0,0.0,1.0,44.0
78,102.000000,0.0,1.0,0.0,0.0,1.0,74.0
